# Μηχανισμοί προσοχής και μετασχηματιστές

Ένα σημαντικό μειονέκτημα των επαναλαμβανόμενων δικτύων είναι ότι όλες οι λέξεις σε μια ακολουθία έχουν την ίδια επίδραση στο αποτέλεσμα. Αυτό προκαλεί υποβέλτιστη απόδοση με τα τυπικά μοντέλα LSTM encoder-decoder για εργασίες ακολουθίας προς ακολουθία, όπως η Αναγνώριση Οντοτήτων και η Μηχανική Μετάφραση. Στην πραγματικότητα, συγκεκριμένες λέξεις στην είσοδο συχνά έχουν μεγαλύτερη επίδραση στα αποτελέσματα από άλλες.

Ας εξετάσουμε ένα μοντέλο ακολουθίας προς ακολουθία, όπως η μηχανική μετάφραση. Αυτό υλοποιείται από δύο επαναλαμβανόμενα δίκτυα, όπου το ένα δίκτυο (**encoder**) συμπυκνώνει την είσοδο σε μια κρυφή κατάσταση, και το άλλο (**decoder**) ξεδιπλώνει αυτή την κρυφή κατάσταση στο μεταφρασμένο αποτέλεσμα. Το πρόβλημα με αυτή την προσέγγιση είναι ότι η τελική κατάσταση του δικτύου δυσκολεύεται να θυμηθεί την αρχή της πρότασης, προκαλώντας έτσι χαμηλή ποιότητα του μοντέλου σε μεγάλες προτάσεις.

**Οι Μηχανισμοί Προσοχής** παρέχουν έναν τρόπο να δίνεται βάρος στην επίδραση κάθε εισόδου στην πρόβλεψη κάθε εξόδου του RNN. Αυτό υλοποιείται δημιουργώντας συντομεύσεις μεταξύ των ενδιάμεσων καταστάσεων του εισόδου RNN και του εξόδου RNN. Με αυτόν τον τρόπο, κατά τη δημιουργία του συμβόλου εξόδου $y_t$, λαμβάνουμε υπόψη όλες τις κρυφές καταστάσεις εισόδου $h_i$, με διαφορετικούς συντελεστές βάρους $\alpha_{t,i}$.

![Εικόνα που δείχνει ένα μοντέλο encoder/decoder με ένα πρόσθετο στρώμα προσοχής](../../../../../translated_images/el/encoder-decoder-attention.7a726296894fb567.webp)
*Το μοντέλο encoder-decoder με μηχανισμό πρόσθετης προσοχής στο [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), από [αυτή την ανάρτηση](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Ο πίνακας προσοχής $\{\alpha_{i,j}\}$ αντιπροσωπεύει τον βαθμό στον οποίο συγκεκριμένες λέξεις εισόδου επηρεάζουν τη δημιουργία μιας δεδομένης λέξης στην ακολουθία εξόδου. Παρακάτω είναι ένα παράδειγμα ενός τέτοιου πίνακα:

![Εικόνα που δείχνει ένα δείγμα ευθυγράμμισης που βρέθηκε από το RNNsearch-50, από το Bahdanau - arviz.org](../../../../../translated_images/el/bahdanau-fig3.09ba2d37f202a6af.webp)

*Εικόνα από [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Οι μηχανισμοί προσοχής είναι υπεύθυνοι για μεγάλο μέρος της τρέχουσας ή σχεδόν τρέχουσας κατάστασης της τέχνης στην Επεξεργασία Φυσικής Γλώσσας. Ωστόσο, η προσθήκη προσοχής αυξάνει σημαντικά τον αριθμό των παραμέτρων του μοντέλου, γεγονός που οδήγησε σε προβλήματα κλιμάκωσης με τα RNNs. Ένας βασικός περιορισμός της κλιμάκωσης των RNNs είναι ότι η επαναλαμβανόμενη φύση των μοντέλων καθιστά δύσκολη την ομαδοποίηση και την παράλληλη εκπαίδευση. Σε ένα RNN κάθε στοιχείο μιας ακολουθίας πρέπει να επεξεργαστεί με τη σειρά, πράγμα που σημαίνει ότι δεν μπορεί να παραλληλιστεί εύκολα.

Η υιοθέτηση μηχανισμών προσοχής σε συνδυασμό με αυτόν τον περιορισμό οδήγησε στη δημιουργία των πλέον κορυφαίων μοντέλων μετασχηματιστών που γνωρίζουμε και χρησιμοποιούμε σήμερα, από το BERT έως το OpenGPT3.

## Μοντέλα μετασχηματιστών

Αντί να προωθούν το πλαίσιο κάθε προηγούμενης πρόβλεψης στο επόμενο βήμα αξιολόγησης, τα **μοντέλα μετασχηματιστών** χρησιμοποιούν **κωδικοποιήσεις θέσης** και προσοχή για να συλλάβουν το πλαίσιο μιας δεδομένης εισόδου μέσα σε ένα παρεχόμενο παράθυρο κειμένου. Η παρακάτω εικόνα δείχνει πώς οι κωδικοποιήσεις θέσης με προσοχή μπορούν να συλλάβουν το πλαίσιο μέσα σε ένα δεδομένο παράθυρο.

![Κινούμενο GIF που δείχνει πώς πραγματοποιούνται οι αξιολογήσεις στα μοντέλα μετασχηματιστών.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Επειδή κάθε θέση εισόδου αντιστοιχίζεται ανεξάρτητα σε κάθε θέση εξόδου, οι μετασχηματιστές μπορούν να παραλληλίζονται καλύτερα από τα RNNs, γεγονός που επιτρέπει πολύ μεγαλύτερα και πιο εκφραστικά γλωσσικά μοντέλα. Κάθε κεφαλή προσοχής μπορεί να χρησιμοποιηθεί για να μάθει διαφορετικές σχέσεις μεταξύ λέξεων, βελτιώνοντας τις εργασίες Επεξεργασίας Φυσικής Γλώσσας.

**BERT** (Bidirectional Encoder Representations from Transformers) είναι ένα πολύ μεγάλο πολυεπίπεδο δίκτυο μετασχηματιστών με 12 επίπεδα για το *BERT-base* και 24 για το *BERT-large*. Το μοντέλο εκπαιδεύεται αρχικά σε μεγάλο σώμα δεδομένων κειμένου (WikiPedia + βιβλία) χρησιμοποιώντας μη επιβλεπόμενη εκπαίδευση (πρόβλεψη λέξεων που έχουν καλυφθεί σε μια πρόταση). Κατά τη διάρκεια της αρχικής εκπαίδευσης, το μοντέλο απορροφά σημαντικό επίπεδο κατανόησης της γλώσσας, το οποίο μπορεί στη συνέχεια να αξιοποιηθεί με άλλα σύνολα δεδομένων μέσω λεπτομερούς προσαρμογής. Αυτή η διαδικασία ονομάζεται **μεταφορά μάθησης**.

![Εικόνα από http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/el/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Υπάρχουν πολλές παραλλαγές αρχιτεκτονικών μετασχηματιστών, όπως BERT, DistilBERT, BigBird, OpenGPT3 και άλλες, που μπορούν να προσαρμοστούν. Το [πακέτο HuggingFace](https://github.com/huggingface/) παρέχει αποθετήριο για την εκπαίδευση πολλών από αυτές τις αρχιτεκτονικές με PyTorch.

## Χρήση του BERT για ταξινόμηση κειμένου

Ας δούμε πώς μπορούμε να χρησιμοποιήσουμε το προεκπαιδευμένο μοντέλο BERT για να λύσουμε την παραδοσιακή μας εργασία: ταξινόμηση ακολουθιών. Θα ταξινομήσουμε το αρχικό μας σύνολο δεδομένων AG News. 

Πρώτα, ας φορτώσουμε τη βιβλιοθήκη HuggingFace και το σύνολο δεδομένων μας:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Επειδή θα χρησιμοποιήσουμε ένα προεκπαιδευμένο μοντέλο BERT, θα χρειαστεί να χρησιμοποιήσουμε έναν συγκεκριμένο tokenizer. Πρώτα, θα φορτώσουμε έναν tokenizer που συνδέεται με το προεκπαιδευμένο μοντέλο BERT.

Η βιβλιοθήκη HuggingFace περιέχει ένα αποθετήριο προεκπαιδευμένων μοντέλων, τα οποία μπορείτε να χρησιμοποιήσετε απλώς καθορίζοντας τα ονόματά τους ως επιχειρήματα στις συναρτήσεις `from_pretrained`. Όλα τα απαραίτητα δυαδικά αρχεία για το μοντέλο θα κατεβούν αυτόματα.

Ωστόσο, σε ορισμένες περιπτώσεις θα χρειαστεί να φορτώσετε τα δικά σας μοντέλα, οπότε μπορείτε να καθορίσετε τον κατάλογο που περιέχει όλα τα σχετικά αρχεία, συμπεριλαμβανομένων των παραμέτρων για τον tokenizer, το αρχείο `config.json` με τις παραμέτρους του μοντέλου, τα δυαδικά βάρη, κ.λπ.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Το αντικείμενο `tokenizer` περιέχει τη συνάρτηση `encode` που μπορεί να χρησιμοποιηθεί άμεσα για την κωδικοποίηση κειμένου:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Στη συνέχεια, ας δημιουργήσουμε επαναλήπτες που θα χρησιμοποιήσουμε κατά τη διάρκεια της εκπαίδευσης για να έχουμε πρόσβαση στα δεδομένα. Επειδή το BERT χρησιμοποιεί τη δική του συνάρτηση κωδικοποίησης, θα χρειαστεί να ορίσουμε μια συνάρτηση συμπλήρωσης παρόμοια με τη `padify` που έχουμε ορίσει προηγουμένως:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

Στην περίπτωσή μας, θα χρησιμοποιήσουμε ένα προεκπαιδευμένο μοντέλο BERT που ονομάζεται `bert-base-uncased`. Ας φορτώσουμε το μοντέλο χρησιμοποιώντας το πακέτο `BertForSequenceClassification`. Αυτό διασφαλίζει ότι το μοντέλο μας έχει ήδη την απαιτούμενη αρχιτεκτονική για ταξινόμηση, συμπεριλαμβανομένου του τελικού ταξινομητή. Θα δείτε ένα προειδοποιητικό μήνυμα που αναφέρει ότι τα βάρη του τελικού ταξινομητή δεν έχουν αρχικοποιηθεί και ότι το μοντέλο θα χρειαστεί προεκπαίδευση - αυτό είναι απολύτως εντάξει, γιατί ακριβώς αυτό πρόκειται να κάνουμε!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Τώρα είμαστε έτοιμοι να ξεκινήσουμε την εκπαίδευση! Επειδή το BERT είναι ήδη προ-εκπαιδευμένο, θέλουμε να ξεκινήσουμε με σχετικά μικρό ρυθμό εκμάθησης, ώστε να μην καταστρέψουμε τα αρχικά βάρη.

Όλη η δύσκολη δουλειά γίνεται από το μοντέλο `BertForSequenceClassification`. Όταν καλούμε το μοντέλο στα δεδομένα εκπαίδευσης, επιστρέφει τόσο την απώλεια (loss) όσο και την έξοδο του δικτύου για το εισαγόμενο minibatch. Χρησιμοποιούμε την απώλεια για τη βελτιστοποίηση των παραμέτρων (το `loss.backward()` εκτελεί το backward pass) και το `out` για τον υπολογισμό της ακρίβειας εκπαίδευσης, συγκρίνοντας τις ετικέτες που προκύπτουν `labs` (υπολογίζονται με χρήση του `argmax`) με τις αναμενόμενες `labels`.

Για να ελέγξουμε τη διαδικασία, συσσωρεύουμε την απώλεια και την ακρίβεια σε αρκετές επαναλήψεις και τις εκτυπώνουμε κάθε `report_freq` κύκλους εκπαίδευσης.

Αυτή η εκπαίδευση πιθανότατα θα διαρκέσει αρκετό χρόνο, οπότε περιορίζουμε τον αριθμό των επαναλήψεων.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Μπορείτε να δείτε (ειδικά αν αυξήσετε τον αριθμό των επαναλήψεων και περιμένετε αρκετά) ότι η ταξινόμηση με BERT μας δίνει αρκετά καλή ακρίβεια! Αυτό συμβαίνει επειδή το BERT ήδη κατανοεί αρκετά καλά τη δομή της γλώσσας, και το μόνο που χρειάζεται είναι να προσαρμόσουμε τον τελικό ταξινομητή. Ωστόσο, επειδή το BERT είναι ένα μεγάλο μοντέλο, όλη η διαδικασία εκπαίδευσης διαρκεί πολύ χρόνο και απαιτεί σημαντική υπολογιστική ισχύ! (GPU, και κατά προτίμηση περισσότερες από μία).

> **Note:** Στο παράδειγμά μας, χρησιμοποιούμε ένα από τα μικρότερα προεκπαιδευμένα μοντέλα BERT. Υπάρχουν μεγαλύτερα μοντέλα που πιθανότατα θα αποδώσουν καλύτερα αποτελέσματα.


## Αξιολόγηση της απόδοσης του μοντέλου

Τώρα μπορούμε να αξιολογήσουμε την απόδοση του μοντέλου μας στο σύνολο δεδομένων δοκιμής. Ο βρόχος αξιολόγησης είναι αρκετά παρόμοιος με τον βρόχο εκπαίδευσης, αλλά δεν πρέπει να ξεχνάμε να αλλάξουμε το μοντέλο σε λειτουργία αξιολόγησης καλώντας `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Συμπέρασμα

Σε αυτήν την ενότητα, είδαμε πόσο εύκολο είναι να πάρουμε ένα προεκπαιδευμένο γλωσσικό μοντέλο από τη βιβλιοθήκη **transformers** και να το προσαρμόσουμε στην εργασία ταξινόμησης κειμένου μας. Παρομοίως, τα μοντέλα BERT μπορούν να χρησιμοποιηθούν για εξαγωγή οντοτήτων, απάντηση ερωτήσεων και άλλες εργασίες επεξεργασίας φυσικής γλώσσας.

Τα μοντέλα Transformer αντιπροσωπεύουν την πιο σύγχρονη τεχνολογία στην επεξεργασία φυσικής γλώσσας, και στις περισσότερες περιπτώσεις θα πρέπει να είναι η πρώτη λύση με την οποία ξεκινάτε να πειραματίζεστε όταν υλοποιείτε προσαρμοσμένες λύσεις NLP. Ωστόσο, η κατανόηση των βασικών αρχών των επαναλαμβανόμενων νευρωνικών δικτύων που συζητήθηκαν σε αυτήν την ενότητα είναι εξαιρετικά σημαντική αν θέλετε να δημιουργήσετε προηγμένα νευρωνικά μοντέλα.



---

**Αποποίηση Ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης AI [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε κάθε προσπάθεια για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν σφάλματα ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
